# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [58]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [59]:
dir_data = './data/'
data_train = os.path.join(dir_data,'titanic_train.csv')
data_test = os.path.join(dir_data,'titanic_test.csv')
df_train = pd.read_csv(data_train)
df_test = pd.read_csv(data_test)

In [60]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [61]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [62]:
train_Y = df_train.Survived
ids = df_test.PassengerId

df_train = df_train.drop(['PassengerId','Survived'], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)

In [63]:
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [64]:
# 只保留數值型資料
num_features = []
for dtype, feature in zip(df.dtypes,df.columns):
    if dtype == 'int64' or dtype =='float64':
        num_features.append(feature)

In [65]:
df = df[num_features]
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [66]:
df.isnull().sum()

Pclass      0
Age       263
SibSp       0
Parch       0
Fare        1
dtype: int64

# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [67]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [68]:
# 空值補 -1, 做 Logistic Regression
df_m1 = df.fillna(-1)
train_num = len(train_Y)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
score_m1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補平均值, 做 Logistic Regression
df_mn = df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LogisticRegression()
score_mn = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 0, 做 Logistic Regression
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
score_0 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('空值補-1       分數為 : ' + str(score_m1))
print('空值補平均值   分數為 : ' + str(score_mn))
print('空值補 0       分數為 : ' + str(score_0))

空值補-1       分數為 : 0.6960299128976762
空值補平均值   分數為 : 0.6981761033723469
空值補 0       分數為 : 0.6971535084032942


c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: 

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [69]:
# 空值補平均值

# 最小最大化
df_min_max = MinMaxScaler().fit_transform(df_mn)
train_X = df_min_max[:train_num]
estimator = LogisticRegression()
score_min_max = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

#標準化
df_standard = StandardScaler().fit_transform(df_mn)
train_X = df_standard[:train_num]
estimator = LogisticRegression()
score_std = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('以  MinMaxScaler  做標準化，平均值填補空值，分數為： ' + str(score_min_max))
print('以 StandardScaler 做標準化，平均值填補空值，分數為： ' + str(score_std))

以  MinMaxScaler  做標準化，平均值填補空值，分數為： 0.6993501991462476
以 StandardScaler 做標準化，平均值填補空值，分數為： 0.6959413955734954


c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\admin\anaconda3\envs\opencv-env\lib\site-packages\sklearn\linear_model\logistic.py:433: Futu